In [1]:
!pip install tensorflow


  Obtaining dependency information for tensorflow-intel==2.16.1 from https://files.pythonhosted.org/packages/e0/36/6278e4e7e69a90c00e0f82944d8f2713dd85a69d1add455d9e50446837ab/tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for tensorboard<2.17,>=2.16 from https://files.pythonhosted.org/packages/3a/d0/b97889ffa769e2d1fdebb632084d5e8b53fc299d43a537acee7ec0c021a3/tensorboard-2.16.2-py3-none-any.whl.metadata
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl (377.0 MB)
Using cached tensorboard-2.16.2-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-i

In [2]:
pip install --upgrade tensorflow


  Obtaining dependency information for tensorflow-intel==2.17.0 from https://files.pythonhosted.org/packages/66/03/5c447feceb72f5a38ac2aa79d306fa5b5772f982c2b480c1329c7e382900/tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Obtaining dependency information for tensorboard<2.18,>=2.17 from https://files.pythonhosted.org/packages/d4/41/dccba8c5f955bc35b6110ff78574e4e5c8226ad62f08e732096c3861309b/tensorboard-2.17.1-py3-none-any.whl.metadata
  Using cached tensorboard-2.17.1-py3-none-any.whl.metadata (1.6 kB)
Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl (385.0 MB)
Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.17.0'

In [5]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow_datasets as tfds


In [31]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
#load mnist dataset
#tfds.load loads the data
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary


{'train': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [8]:
#extract training and test data as it provides only these two not validation data
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [9]:
#now sploit validation data(val) on our own
# we start by defining the number of validation samples as a % of the train samples
num_validation_samples=0.1*mnist_info.splits['train'].num_examples
# convert float to integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [10]:
#store test data in a variable
num_test_samples=mnist_info.splits['test'].num_examples
#again convert into integer
num_test_samples=tf.cast(num_test_samples, tf.int64)

In [11]:
#scale the data to make the result more numerically stable as range of numbers may differ for dif columns
# in this case we will simply prefer to have inputs between 0 and 1
# let's define a function called: scale, that will take an MNIST image and its label
def scale(image,label):
    #value should be float(original data type)
    image=tf.cast(image,tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    image /= 255.
    
    return image,label


In [12]:
# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
scaled_train_and_validation_data = mnist_train.map(scale)

In [13]:
# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
test_data = mnist_test.map(scale)

In [14]:
#we have to shuffle the data because if training data will be sorted the model will not be able to learn on test data
BUFFER_SIZE=10000
# this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets
# then we can't shuffle the whole dataset in one go because we can't fit it all in memory
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them
# if BUFFER_SIZE=1 => no shuffling will actually happen
# if BUFFER_SIZE >= num samples => shuffling is uniform
# BUFFER_SIZE in between - a computational optimization to approximate uniform shuffling

In [15]:
# there is a shuffle method readily available and we just need to specify the buffer size
shuffled_train_and_val_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [16]:
# once we have scaled and shuffled the data, we can proceed to actually extracting the train and validation
# our validation data would be equal to 10% of the training set
# we use the .take() method to take that many samples
# finally, we create a batch with a batch size equal to the total number of validation samples
val_data=shuffled_train_and_val_data.take(num_validation_samples)
# similarly, the train_data is everything else, so we skip as many samples as there are in the validation dataset
train_data=shuffled_train_and_val_data.skip(num_validation_samples)


In [17]:
#as whole dataset can't be passed in once so we create batch sizes
BATCH_SIZE=100
# this would be very helpful when we train, as we would be able to iterate over the different batches
train_data=train_data.batch(BATCH_SIZE)
val_data= val_data.batch(num_validation_samples)
#batch the test data
test_data= test_data.batch(num_test_samples)
# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
val_inputs,val_targets = next(iter(val_data))

In [18]:
#outline the model
input_size=784 #28*28 pixels
output_size=10 #as number starts from 0to9
hidden_layer_size =200 #it is not fixed # we can vary it acc to get more accuracy


In [46]:
#define how the model will look
model=tf.keras.Sequential([
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28,28,1)), #input layer
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [47]:
#optimizer and loss function
# the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [48]:
#training
NUM_EPOCHS=5
# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(val_inputs, val_targets), verbose =2)

Epoch 1/5
540/540 - 9s - 17ms/step - accuracy: 0.9214 - loss: 0.2704 - val_accuracy: 0.9543 - val_loss: 0.1466
Epoch 2/5
540/540 - 4s - 8ms/step - accuracy: 0.9682 - loss: 0.1040 - val_accuracy: 0.9735 - val_loss: 0.0867
Epoch 3/5
540/540 - 4s - 8ms/step - accuracy: 0.9786 - loss: 0.0693 - val_accuracy: 0.9808 - val_loss: 0.0647
Epoch 4/5
540/540 - 5s - 9ms/step - accuracy: 0.9835 - loss: 0.0503 - val_accuracy: 0.9840 - val_loss: 0.0549
Epoch 5/5
540/540 - 6s - 10ms/step - accuracy: 0.9881 - loss: 0.0373 - val_accuracy: 0.9850 - val_loss: 0.0461


In [49]:
#so we will make some adjustments to make the accuracy better
#thus above parameters are adjusted several times and finally decided


In [50]:
#increasing hidden layer size increases the accuracy
#but increasing no of hidden layer takes much time and also didn't increased the accuracy
#for activation i had tried sigmoid (output from 0 to 1) and than (used for exponential) but softmax is much better but sigmoid can also do better sometimes
#reducing batch size increased the time and less accuaracy
#don't do anything with batch size 

In [51]:
#test data
test_loss,test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.9767 - loss: 0.0744


In [52]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07. Test accuracy: 97.67%


In [35]:
#as it doesnt show much difference between val acc and  test acc so data is not overfitted much but in large scale still it can be better